<header style="padding:10px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>VantageCloud Lake Edition Base Data Loading</b></p>
<hr>

In [1]:
# Import the minimal number of packages

import warnings
warnings.filterwarnings('ignore')

import json
import pandas as pd
import teradatasql

from teradataml import *

access_key = ''
secret = ''

# Section 1 - Load Data #

### For this demo, load two BFS Tables and one OFS table ###
1. **Demo.Customers_BFS** Customer Dimension table for banking data set.
2. **Demo.Accounts_Mapping_BFS** Dimension table for mapping Customers to Accounts Fraud Detection data set.
3. **Demo_OFS.Txn_History** Transaction history table for use with the OAF standalone demo.

In [9]:
# /* -------------------------------------------------------- */
# -- Perform this as SYSDBA.
# -- Load demo.Customer_BFS from S3
# /* -------------------------------------------------------- */

# load vars json
with open('vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    
    try:
        cur.execute('DROP TABLE demo.Customers_BFS')
    except Exception as e:
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    
        qry = f'''
        CREATE TABLE demo.Customers_BFS,
        STORAGE = TD_NDSSTORAGE
        AS (
            SELECT
             id,
             customer_identifier,
             firstname,
             lastname,
             email,
             phone,
             birthday,
             streetaddress,
             city,
             state,
             zipcode,
             latitude,
             longitude,
             numaccounts
            FROM
            (
                LOCATION = '/s3/td-demos-data.s3.amazonaws.com/DEMO/CUSTOMERS_EBS/manifest.json'
                AUTHORIZATION = '{{"Access_ID":"{access_key}","Access_Key":"{secret}"}}'
                MANIFEST = 'TRUE'
            ) as D
            ) WITH DATA;
            '''
        
        cur.execute(qry)

OperationalError: [Version 17.20.0.15] [Session 18672] [Teradata Database] [Error 5628] Column numaccounts not found in D or TD_SYSFNLIB.READ_NOS_CONTRACT.
 at gosqldriver/teradatasql.formatError ErrorUtil.go:89
 at gosqldriver/teradatasql.(*teradataConnection).formatDatabaseError ErrorUtil.go:217
 at gosqldriver/teradatasql.(*teradataConnection).makeChainedDatabaseError ErrorUtil.go:233
 at gosqldriver/teradatasql.(*teradataConnection).processErrorParcel TeradataConnection.go:814
 at gosqldriver/teradatasql.(*TeradataRows).processResponseBundle TeradataRows.go:2284
 at gosqldriver/teradatasql.(*TeradataRows).executeSQLRequest TeradataRows.go:851
 at gosqldriver/teradatasql.newTeradataRows TeradataRows.go:708
 at gosqldriver/teradatasql.(*teradataStatement).QueryContext TeradataStatement.go:122
 at gosqldriver/teradatasql.(*teradataConnection).QueryContext TeradataConnection.go:1318
 at database/sql.ctxDriverQuery ctxutil.go:48
 at database/sql.(*DB).queryDC.func1 sql.go:1757
 at datab

In [ ]:
# /* -------------------------------------------------------- */
# -- Perform this as SYSDBA.
# -- Load demo.Accounts_Mapping_BFS from source file
# /* -------------------------------------------------------- */

# load vars json
with open('vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    
    try:
        cur.execute('DROP TABLE Demo.Accounts_Mapping_BFS')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise
            
    
    qry = f'''
    CREATE MULTISET TABLE demo.Accounts_Mapping_BFS
    AS (
        SELECT nameOrig,
         customer_identifier
        FROM
        (
            LOCATION = '/s3/td-demos-data.s3.amazonaws.com/DEMO/ACCOUNTS_MAPPING_EBS/manifest.json'
            AUTHORIZATION = '{{"Access_ID":"{access_key}","Access_Key":"{secret}"}}'
            MANIFEST = 'TRUE'
        ) as D
        ) WITH DATA;
    '''
    cur.execute(qry)
        

In [12]:
# /* -------------------------------------------------------- */
# -- Perform this as SYSDBA.
# -- Create the transaction history OFS table for the OAF demo
# /* -------------------------------------------------------- */

# load vars json
with open('vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    
    try:
        cur.execute('DROP TABLE demo_ofs.Txn_History')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        elif str(e.args).find("4880") >= 1:
            pass
        else:
            raise
            
    
    qry = '''
    CREATE TABLE demo_ofs.Txn_History,

        /*Note the use of STORAGE here to specify OFS
        This can also be set to a user and database default */

        STORAGE=TD_OFSSTORAGE

        /* We can use a SELECT statement to populate the table
        using WITH DATA */
        AS (

            /*Provide json path information to identify the proper fields */
            SELECT Payload.txn_id txn_id,
                Payload.step step,
                Payload."type" "txn_type",
                CAST(Payload.amount AS FLOAT) amount,
                Payload.nameOrig nameOrig,
                CAST(Payload.oldbalanceOrig AS FLOAT) oldbalanceOrig,
                CAST(Payload.newbalanceOrig AS FLOAT) newbalanceOrig,
                Payload.nameDest nameDest,
                CAST(Payload.oldbalanceDest AS FLOAT) oldbalanceDest,
                CAST(Payload.newbalanceDest AS FLOAT) newbalanceDest,
                CAST(Payload.isFraud AS INTEGER) isFraud,
                CAST(Payload.isFlaggedFraud AS INTEGER) isFlaggedFraud

            FROM (
                LOCATION = '/s3/s3.amazonaws.com/trial-datasets/FraudReduction/'
                AUTHORIZATION = repositories.PubAuth
    ) AS D
    ) WITH DATA
    '''
    try:
        cur.execute(qry)
    except Exception as e:
        # Table already exists
        if str(e.args).find("3803") >= 1:
            pass 